In [ ]:
import os
os.getcwd()
from tqdm import tqdm
import numpy as np
import cv2
import random
from sklearn.preprocessing import LabelEncoder
import math

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm
from sklearn.preprocessing import LabelEncoder
import time
import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
DIR = './../dog-breed-identification/train/'

In [ ]:
inception_pb = './inception_pb/inception.pb'
bottleneck_file = './bottleneck/Bottle_neck_values.csv'

In [ ]:
labels_df = pd.read_csv('./labels.csv')
labels_df.head()

In [ ]:
data = []
labels = []
IMG_SIZE = 64
for i, directory in enumerate(tqdm(os.listdir(DIR)[:2000])):
  path1 = os.path.join(DIR, directory)
#   print(path1)
  label = directory.split('.')[0]
#   print(directory)
  img = cv2.imread(path1)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
  labels.append([np.array(labels_df[labels_df['id']==label]['breed'])])
  data.append([np.array(img)])
#   print(labels)
  if i == 2000: break
    
    
    
random.Random(1).shuffle(data)
random.Random(1).shuffle(labels)

In [ ]:
data = np.squeeze(np.array(data))
data.shape

In [ ]:
labels = np.array(labels)
labels = labels.reshape(2000,1)
labels.shape

In [ ]:
label_dff = pd.DataFrame(labels, columns = ['classes'])
# label_unique = label_dff.classes.unique()
en = LabelEncoder()
en.fit(label_dff.classes)
encoded_label = en.transform(label_dff.classes)
label_dff["encoded_label"] = encoded_label
labels = pd.get_dummies(encoded_label)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(data[59])

In [ ]:
labels = np.array(labels)

In [ ]:
data.shape , labels.shape

In [ ]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name="inception")
    return graph

In [ ]:
graph = load_graph(inception_pb)

In [ ]:
def fc_layer(input, size_in, size_out, name="fc", activation = None):
  with tf.name_scope(name):
    w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
    b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
    act = tf.matmul(input, w) + b
    if activation == 'relu' :
        act = tf.nn.relu(act)
    tf.summary.histogram("weights", w)
    tf.summary.histogram("biases", b)
    tf.summary.histogram("activations", act)
    return act

In [ ]:
def compute_cost(logits,Y):
  with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = Y))
    return cost

In [ ]:
with graph.as_default():
    with tf.variable_scope("transfer"):
        bottleneck = graph.get_tensor_by_name('inception/pool_3:0')#From inception graph
        Y = tf.placeholder(tf.float32, [None, 120])
        X = tf.placeholder(tf.float32, [None, 2048])
        #X = bottleneck
        #fc0 = tf.reshape(X,[-1,2048]) #squeeze
        fc1 = fc_layer(X,   2048, 512,  "fc1", activation = 'relu')
        fc2 = fc_layer(fc1, 512,  128,  "fc2", activation = 'relu')
        logits = fc_layer(fc2,   128,  120,     "logits",       activation =  None)
        cost   = compute_cost(logits,Y)
        var = tf.get_collection(key=tf.GraphKeys.TRAINABLE_VARIABLES, scope="transfer/") #trainable var_scope
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001, name="adam").minimize(cost, var_list=var) #var_list

In [ ]:
def create_bottleneck():
    with tf.Session(graph=graph) as sess:
        im_input = graph.get_tensor_by_name('inception/ExpandDims:0')
        Bottle_neck_values = []
        m = data.shape[0]
        indices = np.arange(m)
        for index in tqdm(range(0, m, 1)): # 1 is when SGD
            batch = indices[index:index + 1]
            Bottle_neck_values.append(sess.run(bottleneck, feed_dict= {im_input:data[batch]}))
        df = pd.DataFrame(np.array(np.squeeze(Bottle_neck_values)))
        df.to_csv("./bottleneck/Bottle_neck_values.csv", index=False)
    return Bottle_neck_values

In [ ]:
create_bottleneck()

In [ ]:
def create_placeholder(n_H0, n_W0, n_C0, n_y):
  X = tf.placeholder(tf.float32,[None,n_H0, n_W0, n_C0])
  Y = tf.placeholder(tf.float32,[None,n_y])
  return X,Y

In [ ]:
def conv_layer(input, size_in, size_out, name = 'conv'):
  with tf.name_scope(name):
    w = tf.Variable(tf.truncated_normal([5, 5, size_in, size_out], stddev=0.01), name = 'W')
    b = tf.Variable(tf.constant(0.01, shape =[size_out]), name = 'B')
    conv = tf.nn.conv2d(input, w, strides = [1,1,1,1], padding = 'SAME')
    with tf.name_scope('relu'):
      act = tf.nn.relu(conv + b)
    with tf.name_scope('maxpool'):
        max_pool = tf.nn.max_pool(act, ksize = [1,2,2,1], strides = [1,1,1,1], padding = 'SAME')
  return max_pool

In [ ]:
def iterate_minibatches(inputs,targets,batchsize):#using python operator
  assert inputs.shape[0] == targets.shape[0]
  m = inputs.shape[0]
  indices = np.arange(m)
  np.random.shuffle(indices)
  for index in range(0, m - batchsize + 1, batchsize):
    batch = indices[index:index+batchsize]
    yield inputs[batch], targets[batch]
    
  if m % batchsize != 0:
        batch = indices[math.floor(m/batchsize)*batchsize:m]
        yield inputs[batch], targets[batch]

In [ ]:
bottle_neck = np.array(pd.read_csv("./bottleneck/Bottle_neck_values.csv"))

In [ ]:
bottle_neck.shape , labels.shape

In [ ]:
x_train = bottle_neck[:1500, :]
x_test = bottle_neck[1500:, :]
y_train = labels[:1500, :]
y_test = labels[1500:, :]

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape



In [ ]:
with tf.Session(graph=graph) as sess:
    #sess.run(tf.variables_initializer(var_list = var, name='init'))
    #writer = tf.summary.FileWriter(filename, sess.graph)
    sess.run(tf.global_variables_initializer())

    


    minibatch_size = 2
    num_epochs = 5
    for epoch in range(num_epochs):
        avg_cost = 0
        for i, minibatch in enumerate(iterate_minibatches(x_train, y_train, minibatch_size)):
            (minibatch_X, minibatch_Y) = minibatch
            sess.run(optimizer, feed_dict={X: minibatch_X, Y: minibatch_Y})
            avg_cost += sess.run(cost, feed_dict={X: minibatch_X, Y: minibatch_Y})
            #summ = sess.run(summaryMerged, feed_dict={X: minibatch_X, Y: minibatch_Y})
            #writer.add_summary(summ, i)
        print ("Cost after iteration %i: %f " % (i, avg_cost/(i+1)))
        
        
    print("Training completed!!") 
        



    
    
    with tf.name_scope('accuracy'):
      correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(Y,1))
      accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    
    
    
    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))*100
    train_accuracy = accuracy.eval({X: x_train, Y: y_train})
    test_accuracy = accuracy.eval({X:x_test, Y: y_test})
    print("Train Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)

    
    
    predict_op = tf.argmax(logits, 1) 
    pred = sess.run(predict_op, feed_dict={X: x_test})